In [3]:
import random
import sys

def calc_pi(N):
    M = 0
    for i in range(N):
    # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
    # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N


num_trials = 10**6
#num_trials = int(sys.argv[1])

pi = calc_pi(num_trials)

print("\n \t Computing pi in serial: \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi))

%timeit -r3 calc_pi(num_trials)


 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.143008

398 ms ± 4.58 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [4]:
import sys
import os
import time
import random
import multiprocessing
import numpy as np
from numba import njit

# ==============================================================================
# 1. GESTIÓN DE ARGUMENTOS
# ==============================================================================
num_trials = 10**7 # 10 Millones por defecto

if len(sys.argv) > 1:
    try:
        num_trials = int(sys.argv[1])
    except ValueError:
        pass

# ==============================================================================
# 2. DEFINICIÓN DE FUNCIONES (NÚCLEO DE CÁLCULO)
# ==============================================================================

# --- A) KERNEL LENTO (PYTHON PURO) ---
# Esta función es la que usaría un multiprocessing normal
def kernel_python(n):
    c = 0
    for _ in range(n):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 < 1.0:
            c += 1
    return c

# --- B) KERNEL OPTIMIZADO (NUMBA) ---
# Esta función es la que usaremos en el multiprocessing HÍBRIDO
# 'nogil=True' es vital aquí: indica que esta función no bloquea Python
@njit(nogil=True)
def kernel_numba(n):
    c = 0
    for _ in range(n):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 < 1.0:
            c += 1
    return c

# ==============================================================================
# 3. WORKERS PARA MULTIPROCESSING
# ==============================================================================

# Worker Normal
def worker_pure_mp(n):
    return kernel_python(n)

# Worker Híbrido (Multiprocessing llamando a Numba)
def worker_hybrid_mp(n):
    # Dentro del proceso hijo, ejecutamos código compilado
    return kernel_numba(n)

# Función orquestadora genérica
def run_parallel_simulation(total_trials, n_cores, worker_func):
    # Dividir el trabajo
    trials_per_core = total_trials // n_cores
    chunks = [trials_per_core] * n_cores
    
    # Ajustar resto
    remainder = total_trials % n_cores
    if remainder:
        chunks[-1] += remainder
        
    # Crear Pool y ejecutar
    with multiprocessing.Pool(processes=n_cores) as pool:
        results = pool.map(worker_func, chunks)
        
    total_inside = sum(results)
    return 4.0 * total_inside / total_trials

# ==============================================================================
# 4. EJECUCIÓN PRINCIPAL
# ==============================================================================
if __name__ == '__main__':
    # Leemos configuración de SLURM
    cores_env = int(os.environ.get('OMP_NUM_THREADS', 4))
    
    print(f"\n====== PI MONTE CARLO: ESTRATEGIAS COMBINADAS ({num_trials:.0e} intentos) ======")
    print(f">>> Usando {cores_env} procesos (cores) <<<\n")
    
    # --------------------------------------------------------------------------
    # ESTRATEGIA 1: MULTIPROCESSING ESTÁNDAR (Solo Python)
    # --------------------------------------------------------------------------
    # Sirve de base para ver cuánto mejora al inyectar Numba
    print("--- 1. Multiprocessing Estándar (Python puro en paralelo) ---")
    start = time.time()
    pi_mp = run_parallel_simulation(num_trials, cores_env, worker_pure_mp)
    end = time.time()
    print(f"   Pi: {pi_mp:.6f}")
    print(f"   Tiempo: {end - start:.4f} s\n")

    # --------------------------------------------------------------------------
    # ESTRATEGIA 2: HÍBRIDO (MULTIPROCESSING + NUMBA)
    # --------------------------------------------------------------------------
    # Aquí combinamos la fuerza bruta de MP con la velocidad de Numba
    print(f"--- 2. HÍBRIDO: Multiprocessing + Numba (Optimización Máxima) ---")
    
    # Warm-up (Compilamos la función numba en el proceso principal primero)
    kernel_numba(100) 
    
    start = time.time()
    pi_hybrid = run_parallel_simulation(num_trials, cores_env, worker_hybrid_mp)
    end = time.time()
    
    print(f"   Pi: {pi_hybrid:.6f}")
    print(f"   Tiempo: {end - start:.4f} s")
    print("   -> Comentario: Combinación de procesos aislados ejecutando código máquina.")
    print("========================================================================\n")


====== PI MONTE CARLO: ESTRATEGIAS COMBINADAS (1e+07 intentos) ======
>>> Usando 4 procesos (cores) <<<

--- 1. Multiprocessing Estándar (Python puro en paralelo) ---
   Pi: 3.141833
   Tiempo: 1.0374 s

--- 2. HÍBRIDO: Multiprocessing + Numba (Optimización Máxima) ---
   Pi: 3.141473
   Tiempo: 0.0699 s
   -> Comentario: Combinación de procesos aislados ejecutando código máquina.



# RESULTADOS DE LA EJECUCIÓN: COLA USADA -> MENDEL 

############################################################
    TEST HÍBRIDO (MP + NUMBA) - 100000000 INTENTOS
############################################################
 
>>> CORES: 1 <<<

 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.141064

628 ms ± 534 μs per loop (mean ± std. dev. of 3 runs, 1 loop each)

====== PI MONTE CARLO: ESTRATEGIAS COMBINADAS (1e+08 intentos) ======
>>> Usando 1 procesos (cores) <<<

--- 1. Multiprocessing Estándar (Python puro en paralelo) ---
   Pi: 3.141866
   Tiempo: 64.9770 s

--- 2. HÍBRIDO: Multiprocessing + Numba (Optimización Máxima) ---
   Pi: 3.141744
   Tiempo: 5.6536 s
   -> Comentario: Combinación de procesos aislados ejecutando código máquina.
========================================================================

 
>>> CORES: 4 <<<

 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.142632

637 ms ± 1.43 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

====== PI MONTE CARLO: ESTRATEGIAS COMBINADAS (1e+08 intentos) ======
>>> Usando 4 procesos (cores) <<<

--- 1. Multiprocessing Estándar (Python puro en paralelo) ---
   Pi: 3.141653
   Tiempo: 16.6502 s

--- 2. HÍBRIDO: Multiprocessing + Numba (Optimización Máxima) ---
   Pi: 3.141633
   Tiempo: 1.4307 s
   -> Comentario: Combinación de procesos aislados ejecutando código máquina.
========================================================================

 
>>> CORES: 8 <<<

 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.142032

635 ms ± 1.28 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

====== PI MONTE CARLO: ESTRATEGIAS COMBINADAS (1e+08 intentos) ======
>>> Usando 8 procesos (cores) <<<

--- 1. Multiprocessing Estándar (Python puro en paralelo) ---
   Pi: 3.141532
   Tiempo: 8.2991 s

--- 2. HÍBRIDO: Multiprocessing + Numba (Optimización Máxima) ---
   Pi: 3.141654
   Tiempo: 0.7311 s
   -> Comentario: Combinación de procesos aislados ejecutando código máquina.
========================================================================